In [ ]:
from typing import *
import pandas as pd
from pathlib import Path
import os
import random
import sys
import re
from dotenv import load_dotenv, dotenv_values
from pydriller import Repository
import json
from multiprocessing import Pool
from itertools import chain
from pymongo import MongoClient
import collections

In [ ]:
import dotenv
dotenv.load_dotenv()
ENV = dotenv.dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])

client = MongoClient("localhost", 42692)
db = client.s5_snyk_libio

In [ ]:
# db.mergedVuln.drop()

In [ ]:
libio_vuln_data = list(db.libioVuln.find())
mvn_eco_vuln_data = list(db.mvnEcoVuln.find())
# db.mergedVuln.insert_many(db.libioVuln.find())
# db.mergedVuln.insert_many(db.mvnEcoVuln.find())

In [ ]:
len(libio_vuln_data), len(mvn_eco_vuln_data)

In [ ]:
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in libio_vuln_data)))
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in mvn_eco_vuln_data)))
print(len(set((':'.join(x['vuln_gav'].split(':')[:-1]), x['cve_ref']) for x in libio_vuln_data + mvn_eco_vuln_data)))

In [ ]:
print(len(set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data)))  # 6726  some do not have CVE ID
print(len(set((x['vuln_gav'], x['cve_ref']) for x in mvn_eco_vuln_data)))  # 829
print(len(set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data + mvn_eco_vuln_data)))  # 7490

In [ ]:
libio_group_by_gav_cve = collections.defaultdict(list)
for it in libio_vuln_data:
    libio_group_by_gav_cve[(it['vuln_gav'], it['cve_ref'])].append(it)

mvn_eco_group_by_gav_cve = collections.defaultdict(list)
for it in mvn_eco_vuln_data:
    mvn_eco_group_by_gav_cve[(it['vuln_gav'], it['cve_ref'])].append(it)

In [ ]:
l = list(sorted(libio_group_by_gav_cve.items(), key=lambda x: len(x[1]), reverse=True))
l[0]

In [ ]:
shared_gav_cve_items = list(set(libio_group_by_gav_cve.keys()).intersection(set(mvn_eco_group_by_gav_cve.keys())))
len(shared_gav_cve_items)

In [ ]:
_rk = random.choice(shared_gav_cve_items)
libio_group_by_gav_cve[_rk], mvn_eco_group_by_gav_cve[_rk]  
# 'CVE-2021-37137', 'io.netty:netty-codec:4.1.5.Final'   mvn_eco data has the wrong cve_ref
# 'CVE-2013-7285' mvn_eco data includes 'ReflectionConverter' which is wrong
# 

In [ ]:
import pprint

diff_class_counter = 0

for k in shared_gav_cve_items:
    x = libio_group_by_gav_cve[k]
    if len(x) > 1:
        continue
    x = x[0]
    y = mvn_eco_group_by_gav_cve[k]
    if len(y) > 1:
        continue
    y = y[0]
    
    if set(x['vuln_classes']) == set(y['vuln_classes']):
        continue

    if set(y['vuln_classes']).issubset(set(x['vuln_classes'])):
        continue

    # if set(x['vuln_classes']).issubset(set(y['vuln_classes'])):
    #     continue
        
    diff_class_counter += 1
    pprint.pprint(x)
    print("\n------------\n")
    pprint.pprint(y)
    print("\n@@@@@@@@@@@@@@@\n")

diff_class_counter

have to remove modified classes that are not in the jar (libio)

In [ ]:
mvn_eco_vuln_gav_cve_ref = set((x['vuln_gav'], x['cve_ref']) for x in mvn_eco_vuln_data)
len(mvn_eco_vuln_gav_cve_ref), len(list(x for x in libio_vuln_data if (x['vuln_gav'], x['cve_ref']) not in mvn_eco_vuln_gav_cve_ref))

In [ ]:
libio_vuln_gav_cve_ref = set((x['vuln_gav'], x['cve_ref']) for x in libio_vuln_data)
len(libio_vuln_gav_cve_ref), len(list(x for x in mvn_eco_vuln_data if (x['vuln_gav'], x['cve_ref']) not in libio_vuln_gav_cve_ref))

In [ ]:
# db.mergedVuln.drop()
# db.mergedVuln.insert_many(libio_vuln_data)
# _ = db.mergedVuln.insert_many(x for x in mvn_eco_vuln_data if (x['vuln_gav'], x['cve_ref']) not in libio_vuln_gav_cve_ref)

In [ ]:
uniq_cve = set()
uniq_cve_ga = set()
uniq_cve_gav_set = set()

for it in db.mergedVuln.find():
    cve_ref = it['cve_ref']
    if cve_ref == '':
        cve_ref = it['snyk_url']
    
    assert cve_ref != '' and cve_ref is not None
    
    uniq_cve.add(cve_ref)
    uniq_cve_ga.add((cve_ref, ':'.join(it['vuln_gav'].split(':')[:-1])))
    uniq_cve_gav_set.add((cve_ref, it['vuln_gav']))

print(len(uniq_cve))  
print(len(uniq_cve_ga))
print(len(uniq_cve_gav_set))